In [44]:
import numpy as np
import pandas  as pd
import matplotlib.pyplot as plt
from scipy import stats as s

In [78]:
data_original = pd.read_csv("data.csv")

In [79]:
data_original.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [80]:
data_original.shape

(569, 33)

In [81]:
data_original.drop(columns = ["id" , "Unnamed: 32"] ,inplace = True)

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [82]:
data = data_original.iloc[:,0:3]


In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   diagnosis     569 non-null    object 
 1   radius_mean   569 non-null    float64
 2   texture_mean  569 non-null    float64
dtypes: float64(2), object(1)
memory usage: 13.5+ KB


In [84]:
data[data["diagnosis"] == "B"].shape

(357, 3)

In [85]:
data[data["diagnosis"] == "M"].shape

(212, 3)

In [86]:
training_data_Len = int(0.7 * data.shape[0])
training_data_Len


398

In [87]:
B_training_data = data[data["diagnosis"] == 'B'].iloc[0:training_data_Len//2]

M_training_data = data[data["diagnosis"] == "M"].iloc[0:training_data_Len//2]

In [88]:
B_training_data.shape

(199, 3)

In [89]:
training_data =  pd.concat([B_training_data,M_training_data] ,axis = 0)

In [90]:
training_data

,diagnosis,radius_mean,texture_mean
19,B,13.540,14.36
20,B,13.080,15.71
21,B,9.504,12.44
37,B,13.030,18.42
46,B,8.196,16.84
...,...,...,...
499,M,20.590,21.24
501,M,13.820,24.49
503,M,23.090,19.83
509,M,15.460,23.95


In [91]:
cv_data_len = int(0.2 * data.shape[0])

In [92]:
cv_data_len

113

In [93]:
B_remaining_data = data[data["diagnosis"] == "B"].iloc[training_data_Len//2 : ]
    

M_reamaining_data = data[data["diagnosis"] == "M"].iloc[training_data_Len//2 :]    

In [94]:
B_remaining_data.shape

(158, 3)

In [95]:
M_reamaining_data.shape

(13, 3)

In [96]:
remaining_data = pd.concat([B_remaining_data,M_reamaining_data])

In [97]:
remaining_data

,diagnosis,radius_mean,texture_mean
360,B,12.54,18.07
361,B,13.30,21.57
362,B,12.76,18.84
363,B,16.50,18.29
364,B,13.40,16.95
...,...,...,...
563,M,20.92,25.09
564,M,21.56,22.39
565,M,20.13,28.25
566,M,16.60,28.08


In [98]:
cv_data = remaining_data.iloc[0:cv_data_len]

testing_data = remaining_data.iloc[cv_data_len:]

In [139]:
np.count_nonzero(cv_data.diagnosis =="M")

0

In [99]:
testing_data.shape

(58, 3)

 so we need to evaluate the following prob : 


# P(Diagnosis = M | radiusMean = x) ={ P(radiusMean = x | Diagnosis = M)\dot P(Diaganosis = M)}


In [100]:
mu_hat_rm = training_data[training_data["diagnosis"] == 'M']["radius_mean"].mean()

sigma_hat_rm = training_data[training_data["diagnosis"] == 'M']["radius_mean"].std()

In [101]:
M_prior = training_data[training_data["diagnosis"] == "M"].shape[0] / training_data.shape[0]

B_prior = training_data[training_data["diagnosis"] == "B"].shape[0] / training_data.shape[0]

In [102]:
mu_hat_rb = training_data[training_data["diagnosis"] == 'B']["radius_mean"].mean()

sigma_hat_rb = training_data[training_data["diagnosis"] == 'B']["radius_mean"].std()

In [103]:
def cv_data_mock_test(cv_data):
    
    radius_input = np.array(cv_data["radius_mean"])
    
    posterior_M = s.norm.pdf(radius_input,mu_hat_rm,sigma_hat_rm)*M_prior
    
    posterior_B = s.norm.pdf(radius_input,mu_hat_rb,sigma_hat_rb)*B_prior
    
    boolen_Mask = posterior_M > posterior_B
    
    predicted_cat = pd.Series(boolen_Mask)
    
    predicted_cat.replace(to_replace=[False,True],value = ["B" , "M"] ,inplace = True)
    
    
    return np.array(predicted_cat)

In [104]:
cv_result = cv_data_mock_test(cv_data)
cv_result

array(['B', 'B', 'B', 'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M',
       'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'B', 'M',
       'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'M'], dtype=object)

In [105]:
actual_result = np.array(cv_data["diagnosis"])
actual_result

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'], dtype=object)

In [106]:
boolen_mask = (cv_result == actual_result)
boolen_mask

array([ True,  True,  True, False,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True, False,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False])

In [107]:
accuracy  = np.count_nonzero(boolen_mask)/boolen_mask.shape[0]
accuracy

0.8495575221238938

Now Testing on Testing data and check it`s accuracy

In [108]:
def mock_test(data):
    
    radius_input = np.array(data["radius_mean"])
    
    posterior_M = s.norm.pdf(radius_input,mu_hat_rm,sigma_hat_rm)*M_prior
    
    posterior_B = s.norm.pdf(radius_input,mu_hat_rb,sigma_hat_rb)*B_prior
    
    boolen_Mask = posterior_M > posterior_B
    
    predicted_cat = pd.Series(boolen_Mask)
    
    predicted_cat.replace(to_replace=[False,True],value = ["B" , "M"] ,inplace = True)
    
    
    return np.array(predicted_cat)

In [109]:
testing_result = mock_test(testing_data)

In [110]:
actual_testing_data = np.array(testing_data["diagnosis"])

In [111]:
testing_result

array(['B', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'B', 'B', 'M', 'M', 'M', 'M', 'M', 'M', 'B',
       'M', 'M', 'M', 'M', 'M', 'M'], dtype=object)

In [112]:
boolen_mask_testing = (testing_result == actual_testing_data )

In [113]:
testing_accuracy = np.count_nonzero(boolen_mask_testing)/testing_data.shape[0]

testing_accuracy

0.896551724137931

In [114]:
data = data.iloc[:,0:4]
data

,diagnosis,radius_mean,texture_mean
0,M,17.99,10.38
1,M,20.57,17.77
2,M,19.69,21.25
3,M,11.42,20.38
4,M,20.29,14.34
...,...,...,...
564,M,21.56,22.39
565,M,20.13,28.25
566,M,16.60,28.08
567,M,20.60,29.33


\begin{equation}
P(Dia = M|radiusMean,textureMean = x) = P(radiusMean = x | Dia = M)\dot P(textureMean = x | Dia = M)\dot P(Dia = M)
\end{equation}

In [120]:
mu_hat_b = training_data[training_data["diagnosis"] == 'B'].iloc[1:].mean()
mu_hat_b

radius_mean     11.976540
texture_mean    17.124798
dtype: float64

In [124]:
sigma_hat_b = training_data[training_data["diagnosis"] == 'B'].iloc[:,1:].cov()
sigma_hat_b_cov = np.array(sigma_hat_b)

In [127]:
mu_hat_m = np.array(training_data[training_data["diagnosis"] == 'M'].iloc[1:].mean())
mu_hat_m

array([17.35318182, 21.51772727])

In [129]:
sigma_hat_m = training_data[training_data["diagnosis"] == 'M'].iloc[:,1:].cov()
sigma_hat_m_cov = np.array(sigma_hat_b)
sigma_hat_b_cov

array([[ 2.94091972,  0.15193868],
       [ 0.15193868, 11.32885473]])

In [130]:
def multivarate_prob(data):
    
    inputs = np.array(data.iloc[:,1:3])
    
    posterior_m = s.multivariate_normal.pdf(inputs,mu_hat_m,sigma_hat_m) * M_prior
    
    posterior_b  = s.multivariate_normal.pdf(inputs,mu_hat_b,sigma_hat_b) *B_prior
    
    boolen_mask = posterior_b > posterior_m
    
    predicted_cat = pd.Series(boolen_mask)
    
    predicted_cat.replace(to_replace=[False,True] ,value=["M" ,"B"],inplace= True)
    
    return np.array(predicted_cat)

In [131]:
cv_result_multi = multivarate_prob(cv_data)

In [140]:
actual_result_multi = np.array(cv_data["diagnosis"])

In [141]:
actual_result_multi

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'], dtype=object)

In [142]:
cv_accuracy_multi = np.count_nonzero(cv_result_multi == actual_result_multi)/actual_result.shape[0]

cv_accuracy_multi

0.831858407079646

In [ ]:
testing_ac